# 작물 잎 사진으로 질병 분류하기

In [1]:
import os
import shutil

## 데이터 분할을 위한 폴더 생성

In [2]:
original_dataset_dir = './dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
os.mkdir(base_dir)

In [3]:
train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

for clss in classes_list:
    os.mkdir(os.path.join(train_dir,clss))
    os.mkdir(os.path.join(validation_dir,clss))
    os.mkdir(os.path.join(test_dir,clss))

## 데이터 분할과 클래스별 데이터 수 확인

In [17]:
import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir,clss)
    fnames = os.listdir(path)
    
    train_size = math.floor(len(fnames)* 0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)
    
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'):',len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(os.path.join(train_dir,clss),fname)
        shutil.copyfile(src,dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'):',len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(os.path.join(validation_dir,clss),fname)
        shutil.copyfile(src,dst)
    
    test_fnames = fnames[(train_size + validation_size):(validation_size + train_size + test_size)]
    print('Test size(',clss,'):',len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(os.path.join(test_dir,clss),fname)
        shutil.copyfile(src,dst)

Train size( Pepper,_bell___healthy ): 886
Validation size( Pepper,_bell___healthy ): 295
Test size( Pepper,_bell___healthy ): 295
Train size( Pepper,_bell___Bacterial_spot ): 598
Validation size( Pepper,_bell___Bacterial_spot ): 199
Test size( Pepper,_bell___Bacterial_spot ): 199
Train size( Grape___Leaf_blight_(Isariopsis_Leaf_Spot) ): 645
Validation size( Grape___Leaf_blight_(Isariopsis_Leaf_Spot) ): 215
Test size( Grape___Leaf_blight_(Isariopsis_Leaf_Spot) ): 215
Train size( Cherry___healthy ): 512
Validation size( Cherry___healthy ): 170
Test size( Cherry___healthy ): 170
Train size( Tomato___Tomato_mosaic_virus ): 223
Validation size( Tomato___Tomato_mosaic_virus ): 74
Test size( Tomato___Tomato_mosaic_virus ): 74
Train size( Tomato___healthy ): 954
Validation size( Tomato___healthy ): 318
Test size( Tomato___healthy ): 318
Train size( Apple___Apple_scab ): 378
Validation size( Apple___Apple_scab ): 126
Test size( Apple___Apple_scab ): 126
Train size( Corn___Northern_Leaf_Blight )

## 베이스라인 모델 학습을 위한 준비

In [5]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE =  256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

train_dataset = ImageFolder(root = './splitted/train',transform = transform_base)
val_dataset = ImageFolder(root = './splitted/val',transform = transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle = True,num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle = True,num_workers=4)

## 베이스라인 모델 설계

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        
        super(Net,self).__init__()
        
        self.conv1 = nn.Conv2d(3,32,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.conv3 = nn.Conv2d(64,64,3,padding=1)
        
        self.fc1 = nn.Linear(4096,512)
        self.fc2 = nn.Linear(512,33)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p = 0.25,training = self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p = 0.25,training = self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p = 0.25,training = self.training)
        
        x = x.view(-1,4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p = 0.5,training = self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x,dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(),lr=0.001)

## 모델 학습을 위한 함수

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = data.to(DEVICE),target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

## 모델 평가를 위한 함수

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 
    correct = 0 
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output,target,reduction='sum').item()
            
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            ##target.view_as(pred)를 통해 target Tensor의 구조를 pred Tensor와 같은 모양으로 정렬합니다. 
            ##view_as() 메서드는 적용 대상 Tensor를 메서드에 입력되는 Tensor의 모양대로 재정렬하는 함수입니다. 
            ##앞서 모델 구조를 구성할 때 학습했던 view() 함수는 정렬하고 싶은 Tensor의 모양을 숫자로 직접 지정한다는 점에서 차이가 있습니다.
            ## eq() 메서드는 객체 간의 비교 연산자로, 
            ##pred.eq(target.view_as(pred))은 pred와 target,view_as(pred)의 값이 일치하면 1, 일치하지 않으면 0을 반환합니다. 
            
    test_loss /= len(test_loader.dataset)
    test_accuracy =  100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

## 모델 학습 실행하기

In [9]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model,train_loader,optimizer)
        train_loss,train_acc = evaluate(model,train_loader)
        val_loss,val_acc = evaluate(model,val_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time()-since
        
        print("----------- epoch {} ----------------".format(epoch))
        print('train Loss:{:.4f}, Accuracy:{:.2f}%'.format(train_loss,train_acc))
        print('val Loss:{:.4f}, Accuracy:{:.2f}%'.format(val_loss,val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader,val_loader, optimizer, EPOCH)

torch.save(base,'baseline.pt')

----------- epoch 1 ----------------
train Loss:2.0548, Accuracy:43.65%
val Loss:2.0551, Accuracy:44.01%
Completed in 0m 25s
----------- epoch 2 ----------------
train Loss:1.3077, Accuracy:61.02%
val Loss:1.3239, Accuracy:60.46%
Completed in 0m 24s
----------- epoch 3 ----------------
train Loss:1.1548, Accuracy:64.49%
val Loss:1.1758, Accuracy:64.05%
Completed in 0m 25s
----------- epoch 4 ----------------
train Loss:0.8720, Accuracy:73.69%
val Loss:0.9108, Accuracy:72.02%
Completed in 0m 25s
----------- epoch 5 ----------------
train Loss:0.7447, Accuracy:76.92%
val Loss:0.7819, Accuracy:75.56%
Completed in 0m 25s
----------- epoch 6 ----------------
train Loss:0.6381, Accuracy:80.50%
val Loss:0.6823, Accuracy:78.88%
Completed in 0m 25s
----------- epoch 7 ----------------
train Loss:0.5847, Accuracy:81.94%
val Loss:0.6393, Accuracy:79.86%
Completed in 0m 25s
----------- epoch 8 ----------------
train Loss:0.4905, Accuracy:85.56%
val Loss:0.5495, Accuracy:82.85%
Completed in 0m 25s


## Transfer Learning을 위한 준비

In [10]:
data_transforms ={
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = './splitted'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir,x),
                                transform = data_transforms[x]) for x in ['train','val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                             batch_size = BATCH_SIZE, shuffle=True, num_workers= 4)
                                              for x in ['train','val']}
dataset_size = {x: len(image_datasets[x]) for x in ['train','val']}

class_names = image_datasets['train'].classes

## Pre-Trained Model 불러오기

In [11]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
##pretrained = True로 설정하면 미리 학습된 모델의 Parameter 값을 그대로 가져옵니다. 
## prarameter 값을 가져가면 이미 ImageNet에서 학습했던 Weight 값이 있어서 수렴속도가 빠르게 진행 될 수 있다.
## 어떤 특정한 분포에서 가중치를 초기화하면서 진행하는 것이 더 나은 편이다. 
##pretrained = False로 설정하면 모델의 구조만을 가져오고, Parameter 값은 랜덤으로 설정됩니다. 

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 33)
## 이 프로젝트에서는 데이터를 33개의 클래스로 분류해야 하기 때문에 모델의 마지막 Layer의 출력 채널 수는 33개여야합니다.
## 하지만 우리가 불러온 ResNet50 모델은 다른 주제를 위해 설계되었기에 마지막 layer의 출력 채널 수가 33개가 아닙니다. 
## 따라서 불러온 모델을 이 프로젝트의 주제에 맞추고자 모델의 마지막 Fully Conneted Layer 대신 출력 채널의 수가 33개인
## 새로운 Layer를 추가할 것입니다. 이를 위해 불러온 ResNet50에서 마지막 Layer의 입력 채널의 수를 저장합니다. 
## in_features 는 해당 Layer의 입력 채널 수를 의미합니다. 
## 불러온 모델의 마지막 Fully Connected Layer를 새로운 Layer로 교체합니다. 
## 이때, 입력 채널의 수는 기본의 layer의 입력 채널 수를 의미합니다.

resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft= optim.Adam(filter(lambda p : p.requires_grad,
                               resnet.parameters()),lr=0.001)
## 앞서 학습한 베이스라인 모델에서는 모든 Parameter를 업데이트했지만 
## 이 모델에서는 설정한 일부 Layer의 Parameter만을 업데이트해야 합니다. 
## 따라서 filter() 메서드와 lambda 표현식을 사용하여 requires_grad = True로 설정된 layer의 Parameter에만 적용합니다. 

from torch.optim import lr_scheduler

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)

## 7 Epoch 마다 0.1씩 곱해 Learning Rate를 감소시킨다는 의미 

## Pre-Trained Model의 일부 Layer Freeze하기 

In [12]:
ct = 0 
for child in resnet.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False 

In [13]:
resnet.children()
## Resnet 모델의 모든 layer 정보를 담고 있음 

<generator object Module.children at 0x7f48a8e4aac0>

##  Transfer Learning 모델 학습과 검증을 위한 함수

In [14]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('-----------epochs{}----------'.format(epoch+1))
        since = time.time()
        
        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
        
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                ## 학습 단계에서만 모델의 gradient를 업데이트하고, 검증 단계에서는 업데이트하지 않아야 함
                ## 따라서 set_grad_enabled() 메서드를 이용하여 phase를 train일 경우에만 업데이트하도록 설정
                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1) ## 가장 높은 값을 가진 인덱스를 예측값으로 저장 
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    ## 모델이 현재 학습모드인 경우, 
                    ##위에서 계산한 Loss 값을 바탕으로 Back Propagation을 통해 계산한 Gradient값을
                    ##각 Parameter에 할당하고, 모델의 Parameter를 업데이트 함
                    
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                
                scheduler.step()
                
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                ## optimizer_ft.param_groups의 원소는 학습과정에서의 Parameter를 저장하고 있는 딕셔너리
        
                print('learning_rate : ',l_r)
                
            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double()/dataset_size[phase]
            
            print('{} Loss : {:.4f} Acc : {:.4f}'.format(phase, epoch_loss,epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    print('Best val Acc : {:.4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model

## 모델 학습 실행하기 

In [15]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs = EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

-----------epochs1----------
learning_rate :  [0.001]
train Loss : 0.5950 Acc : 0.8201
val Loss : 0.3667 Acc : 0.8851
Completed in 0m 17s
-----------epochs2----------
learning_rate :  [0.001]
train Loss : 0.2332 Acc : 0.9239
val Loss : 0.2198 Acc : 0.9327
Completed in 0m 17s
-----------epochs3----------
learning_rate :  [0.001]
train Loss : 0.1715 Acc : 0.9447
val Loss : 0.2297 Acc : 0.9282
Completed in 0m 18s
-----------epochs4----------
learning_rate :  [0.001]
train Loss : 0.1503 Acc : 0.9516
val Loss : 0.1593 Acc : 0.9499
Completed in 0m 18s
-----------epochs5----------
learning_rate :  [0.001]
train Loss : 0.1195 Acc : 0.9619
val Loss : 0.1273 Acc : 0.9567
Completed in 0m 17s
-----------epochs6----------
learning_rate :  [0.001]
train Loss : 0.0955 Acc : 0.9689
val Loss : 0.1664 Acc : 0.9473
Completed in 0m 17s
-----------epochs7----------
learning_rate :  [0.0001]
train Loss : 0.0905 Acc : 0.9698
val Loss : 0.1081 Acc : 0.9664
Completed in 0m 17s
-----------epochs8----------
lear

## 베이스라인 모델 평가를 위한 전처리

In [18]:
transform_base = transforms.Compose([transforms.Resize([64,64]), transforms.ToTensor()])

test_base = ImageFolder(root = './splitted/test', transform = transform_base)

test_loader_base = torch.utils.data.DataLoader(test_base, batch_size = BATCH_SIZE, shuffle= True, num_workers = 4)

## Transfer learning 모델 평가를 위한 전처리

In [25]:
transform_resNet= transforms.Compose([
    transforms.Resize([64,64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_resNet = ImageFolder(root='./splitted/test', transform = transform_resNet)

test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size= BATCH_SIZE, shuffle = True, num_workers=4)

## 베이스 라인 모델 성능 평가하기

In [22]:
baseline = torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc: ', test_accuracy)

baseline test acc:  93.39091250469396


## Transefer learning 모델 성능 평가하기 

In [26]:
resnet50 = torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc:', test_accuracy)

ResNet test acc: 99.04869195143323
